## Step 1 - Loading the dependencies

On this notebook, we will generate item-item recommendations using a technique called https://en.wikipedia.org/wiki/Collaborative_filtering. Let's get started!

In [3]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Step 2: Load the Data

Let's download a small version of the MovieLens dataset. You can access it via the zip file url http://files.grouplens.org/datasets/movielens/ml-latest-small.zip, or directly download here. We're working with data in ml-latest-small.zip and will need to add the following files to our local directory:


In [4]:
ratings = pd.read_csv("data/ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:

movies = pd.read_csv("h/movies.csv")
movies.head()